#  Data management: Conversion and integration of csv files on flights, airports and airlines into json files

In [ ]:
import pandas as pd  # for csv reader
import datetime 
import numpy as np  # for checking NaN values
import json
import jsonschema
from jsonschema import validate, Draft7Validator # pip install --upgrade jsonschema

## Create validator for json files

In [ ]:
def json_validator(json_schema, json_data):
  """
    validateJson() validates a json file against a proper schema


    :param json_schema: schema file for validating json data file
    :param json_data: file json to validate
    :return: True if file is valid against schema, False otherwise
  """ 

  try:
    Draft7Validator(json_schema).validate(json_data)
  except jsonschema.exceptions.ValidationError as err:
    return False
  return True

## Data loading and preparation 

In [ ]:
def is_NaN(val):
  """
    is_NaN() check if a value is null for string and float types

    :param val: value to check
    :return: True if value is NaN or NaT or null, False otherwise
  """ 

  # check string NaN
  if isinstance(val, str) and not (val == '' or pd.isnull(val)):
    return False
  # check NaT/NaN value for date
  elif not pd.isnull(val):
    return False

  return True

#### Airlines data

In [ ]:
# get confidence with airline data
temp = pd.read_csv('./airlines.csv')
temp.head()

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N


In [ ]:
def create_airlines_dictionary():
  """
    create_airlines_dictionary() loads airline data and create a key-value 
    dictionary containing IATA-name pairs

    :return: Dictionary contains pairs with IATA codes and airline's name
  """ 

  # reading airline data 
  data_airlines = pd.read_csv('./airlines.csv')

  airlines_dictionary = {}
  # for each line if IATA code is valid create a pair with airline's name
  # key = airline's IATA code, value = airline's name
  for line in data_airlines.itertuples(index=False): 
    if not is_NaN(line.IATA) and line.IATA != "-":
      airlines_dictionary[line.IATA.upper()] = line.Name
  
  return(airlines_dictionary)

#### Airports data

In [ ]:
# get confidence with airport data
temp = pd.read_csv('./airports.csv')
temp.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [ ]:
def create_airport_dictionary():
  """
    create_airport_dictionary() loads airport data and create a key-value 
    dictionary which contains key value pairs where:
    key: airport's iata code
    value: list of values as follow [icao code, name, airport type, 
                                     elevation_ft, latitude, longitude]

    :return: Dictionary contains pairs with IATA codes and a list of information
             for the airport
  """ 

  # reading airline data 
  data_airport = pd.read_csv('./airports.csv')

  airport_dictionary = {}
  # for each line if IATA code is valid create a pair with airport's info
  # key = airport's IATA code
  # value: [icao, name, type, elevation_ft, latitude, longitude]
  for line in data_airport.itertuples(index=False): 
    if not is_NaN(line.iata_code):
      longi, lati = line.coordinates.split(",", 1)
      airport_dictionary[line.iata_code.upper()] = [line.gps_code, line.name,
                                               line.type, int(line.elevation_ft),
                                               float(lati), float(longi)]
  
  return(airport_dictionary)

#### Top 50 airports passengers data

In [ ]:
# get confidence with passengers data
col_names=['world_ranking', 'USA_ranking', 'country', 'city_state', 
           'IATA', 'tot_passengers', 'growth_percent'] 

temp = pd.read_excel('Passengers_2019.xlsx',header=2, names=col_names)
temp.head()

,world_ranking,USA_ranking,country,city_state,IATA,tot_passengers,growth_percent
0,1,1,United States,Atlanta GA,ATL,110531300,0.029213
1,3,2,United States,Los Angeles CA,LAX,88068013,0.006110
2,6,3,United States,Chicago IL,ORD,84649115,0.016861
3,10,4,United States,Dallas/Fort Worth TX,DFW,75066956,0.086154
4,16,5,United States,Denver CO,DEN,69015703,0.070100


In [ ]:
def create_passengers_dictionary():
  """
    create_passengers_dictionary() loads airport passengers data and create a 
    key-value dictionary which contains key value pairs where:
    key: airport's iata code
    value: list of values as follow ["year;world_ranking;tot_passengers;growth_percent", ...]

    :return: Dictionary contains pairs with IATA codes and a list of passengers 
             data for the airport
  """ 

  # define column names for the dataFrame 
  col_names=['world_ranking', 'USA_ranking', 'country', 'city_state', 'IATA', 
            'tot_passengers', 'growth_percent'] 


  airport_passengers_dict = {}
  # read passengers data from file for 2018 and 2019
  for i in range(2018,2020):
    # reading excel data
    data_ap = pd.read_excel('Passengers_' + str(i) + '.xlsx',
                                            header=2, names=col_names)
    
    # For each line in the file, it adds the information to the dictionary 
    for line in data_ap.itertuples(index=False): 
      # corret growth info, because some file is in % and others not
      if line[6] < 0.1:
        growth = line[6]*100
      else:
        growth = line[6]

      # create string by concatenate information
      to_add = str(i) + ";" + str(line[0]) + ";" + str(line[5]) + ";" + str(growth)
      # get airport iata code
      airport_iata = line[4].upper()

      # check if the code already exist and append or create the values for the key
      if airport_iata in airport_passengers_dict.keys():
        (airport_passengers_dict[airport_iata]).append(to_add)
      else:
        airport_passengers_dict[airport_iata] = [to_add]
      

  return airport_passengers_dict

#### Flights data

In [ ]:
# get confidence with data
temp = pd.read_csv("./2018_1.csv", usecols=list(range(0, 32)))
temp.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_NM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_GROUP,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_GROUP,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2018,1,14,7,9E,SYR,"Syracuse, NY",New York,DTW,"Detroit, MI",Michigan,535,NaN,NaN,NaN,735,NaN,NaN,NaN,1.0,B,0.0,120.0,NaN,NaN,374.0,2,NaN,NaN,NaN,NaN,NaN
1,2018,1,3,3,9E,SYR,"Syracuse, NY",New York,LGA,"New York, NY",New York,1358,1348.0,-10.0,-1.0,1519,1506.0,-13.0,-1.0,0.0,NaN,0.0,81.0,78.0,42.0,198.0,1,NaN,NaN,NaN,NaN,NaN
2,2018,1,6,6,9E,SYR,"Syracuse, NY",New York,LGA,"New York, NY",New York,1358,1410.0,12.0,0.0,1519,1543.0,24.0,1.0,0.0,NaN,0.0,81.0,93.0,45.0,198.0,1,12.0,0.0,12.0,0.0,0.0
3,2018,1,7,7,9E,SYR,"Syracuse, NY",New York,LGA,"New York, NY",New York,1358,1347.0,-11.0,-1.0,1519,1455.0,-24.0,-2.0,0.0,NaN,0.0,81.0,68.0,38.0,198.0,1,NaN,NaN,NaN,NaN,NaN
4,2018,1,8,1,9E,SYR,"Syracuse, NY",New York,LGA,"New York, NY",New York,1358,1350.0,-8.0,-1.0,1519,1509.0,-10.0,-1.0,0.0,NaN,0.0,81.0,79.0,39.0,198.0,1,NaN,NaN,NaN,NaN,NaN


In [ ]:
def load_flight_csv(filepath):
  """
    load_flight_csv() Read the flight data files and prepare the dataset 
    for processing by converting the data fields to the right type 

    :param filepath: path from which to read the data 
    :return: Pandas dataframe with data
  """ 
  
  # read data from file
  flight_data = pd.read_csv(filepath, usecols=list(range(0, 32)))
  
  # I try to convert time to string here but when I read line by line I lost 
  # the convertion so I done it later!
  
  return flight_data

## Data management: Creating JSON files and uploading to mongoDB 
Each files groups year, month, origin airport, destination airport and day of the week <br>
(**NOTE:** year and month are grouped automatically as the files are already divided!) 

### Create json section about airports info

In [ ]:
# Init dictionaries for airlines, airports and passengers data
airline_iata_name_dic = create_airlines_dictionary()
airport_data_dic = create_airport_dictionary()
airport_passengers_dic = create_passengers_dictionary()

In [ ]:
def json_airport_info(year, airport_code, str_city_state, str_state_name):
  """
    json_airport_info() create a python dictionary with all the information 
    of a specific airport.

    :param year: Reference year to eventually enter the list with passengers data 
    :param airport_code: IATA code of the airport
    :param str_city_state: String consisting of "CityName, stateCode"
    :param str_state_name: String consisting of "StateName"  
    :return: Python dictionary with the followind data:
             {ICAO, name, type, elevation_ft, latitude, longitude, city_name,
             state_name, state_code}
  """
  airport_iata = airport_code.upper()

  # get all info from airport dictionary, I'm sure the airport code exists in the 
  # dictionary as a key otherwise I'd have to use a try-catch or an if statement
  info_list = airport_data_dic[airport_iata]

  # split city name and state code
  city, state_code = str_city_state.split(",", 1)

  num_pass = []
  # Check if the we have info about passengers for this airport
  if airport_iata in airport_passengers_dic.keys():
    single_pass = {}
    for str_val in airport_passengers_dic[airport_iata]:
      # split string of values
      actual_year, rank, tot_pass, growth = str_val.split(";",4)

      # If the year is <= the reference year I add it 
      # (i.e. If I am considering a 2018 flight I do not add info on 2019 passengers)
      if int(actual_year) <= int(year):
        single_pass["year"] = int(actual_year)
        single_pass["world_ranking"] = int(rank)
        single_pass["total_passengers"] = int(tot_pass)
        single_pass["growth_percent"] = float(growth)
    num_pass.append(single_pass)

  # create dictionary with airport info
  airport_info = {
      "ICAO": info_list[0],
      "name": info_list[1],
      "type": info_list[2],
      "elevation_ft": info_list[3],
      "latitude": info_list[4],
      "longitude": info_list[5],
      "city_name": city,
      "state_name": str_state_name,
      "state_code": state_code.strip() # delete space
  }

  # If information exist I add the array with the info about passengers
  if num_pass != []:
    airport_info["num_passengers"] = num_pass

  return airport_info

### Create json section about flights

In [ ]:
def convert_str_to_time(str_to_convert):
  """
    convert_str_to_time() Convert a string to time format. 
    Example: input = 1210   return = 12:10

    :param str_to_convert: String to convert into time format
    :return: String in format time
  """ 

  # reading time as string to simplify casting, I can not use pd.to_datetime
  # because I have midnight as 24:00 and not as 00:00 
  str_to_convert = str(str_to_convert)
  if len(str(str_to_convert)) == 4:
    if str_to_convert == "2400":
      return "00:00"
    return str_to_convert[0:2] + ":" + str_to_convert[2:]
  
  return "00" + ":" + str_to_convert

In [ ]:
def json_single_flight(fligth_data):
  """
    json_single_flight() create a python dictionary with all the information 
    of a specific flight. Only valid values will be added to the dictionary
    (i.e. NaN/null/NaT values will not be add, refers to db schema)

    :param fligth_data: flight data to parse
    :return: Python dictionary of the data
  """ 

  time_dic = {}

  # ailine's IATA code
  airline_IATA_code = fligth_data.OP_CARRIER.upper()
  time_dic['airline'] = airline_IATA_code
   
  # get the airline name from IATA codes by using the dictionary init before
  if airline_IATA_code in airline_iata_name_dic.keys():
    time_dic['airline_name'] = airline_iata_name_dic[airline_IATA_code]

  # departure time
  time_dic['crs_dep_time'] = convert_str_to_time(fligth_data.CRS_DEP_TIME)

  # saving departure time
  if not is_NaN(fligth_data.DEP_TIME):
    time_dic['dep_time'] = convert_str_to_time(fligth_data.DEP_TIME)

  # saving delay as integer, cast will be always safe thanks to if
  if not is_NaN(fligth_data.DEP_DELAY):
    time_dic['dep_delay'] = int(fligth_data.DEP_DELAY)

  # saving dep delay group as integer, cast will be always safe thanks to if
  if not is_NaN(fligth_data.DEP_DELAY_GROUP):
    time_dic['dep_delay_group'] = int(fligth_data.DEP_DELAY_GROUP)

  # arrival time (Does not contains NaN)
  time_dic['crs_arr_time'] = convert_str_to_time(fligth_data.CRS_ARR_TIME)

  # saving arrival time
  if not is_NaN(fligth_data.ARR_TIME):
    time_dic['arr_time'] = convert_str_to_time(fligth_data.ARR_TIME)

  # saving delay as integer, cast will be always safe thanks to if
  if not is_NaN(fligth_data.ARR_DELAY):
    time_dic['arr_delay'] = int(fligth_data.ARR_DELAY)

  # saving arrival delay group as integer, cast will be always safe thanks to if
  if not is_NaN(fligth_data.ARR_DELAY_GROUP):
    time_dic['arr_delay_group'] = int(fligth_data.ARR_DELAY_GROUP)

  # check if flight was cancelled (No NaN)
  if fligth_data.CANCELLED == 1.0:
    time_dic['cancelled'] = True
    time_dic['cancellation_code'] = fligth_data.CANCELLATION_CODE
  else:
    time_dic['cancelled'] = False

  # check if flight was diverted
  if fligth_data.DIVERTED == 1.0:
    time_dic['diverted'] = True
  else:
    time_dic['diverted'] = False

  # saving crs Elapsed Time of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.CRS_ELAPSED_TIME):
    time_dic['crs_elapsed_time'] = int(fligth_data.CRS_ELAPSED_TIME)

  # saving real Elapsed Time of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.ACTUAL_ELAPSED_TIME):
    time_dic['actual_elapsed_time'] = int(fligth_data.ACTUAL_ELAPSED_TIME)

  # saving air Time of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.AIR_TIME):
    time_dic['air_time'] = int(fligth_data.AIR_TIME)

  # saving carrier_delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.CARRIER_DELAY):
    time_dic['carrier_delay'] = int(fligth_data.CARRIER_DELAY)

  # saving weather delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.WEATHER_DELAY):
    time_dic['weather_delay'] = int(fligth_data.WEATHER_DELAY)

  # saving nas delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.NAS_DELAY):
    time_dic['nas_delay'] = int(fligth_data.NAS_DELAY)

  # saving security delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.SECURITY_DELAY):
    time_dic['security_delay'] = int(fligth_data.SECURITY_DELAY)

  # saving late aircraft delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.LATE_AIRCRAFT_DELAY):
    time_dic['late_aircraft_delay'] = int(fligth_data.LATE_AIRCRAFT_DELAY)	

  return time_dic

### Create json basic structure for each .csv files

In [ ]:
# TODO: va messo un for che per ogni file presente nella cartella esegue i seguenti passi:
# 1) load (metodo gia implementato)
# 2) group + creazione + upload (già implementati)

# reading data on flights 
data = load_flight_csv('./2018_1.csv')

In [ ]:
# group data, each file already group for year and month to!
data_groups = data.groupby(['DAY_OF_WEEK', 'ORIGIN', 'DEST'])

In [ ]:
# FOR TESTING VERSION uncomment the follow line
# i = 0

# Create a json file and upload it to mongo database for each group of 
# day-origin-destination (Remember: each file contains a specific month and year)
for group in data_groups:
  document = {}
  # list of all flights for a specific day, origin and destination
  flights = []
  for flight in group[1].itertuples():
    # Add basic information in case it is a new group
    if document == {}:
      document['year'] = flight.YEAR
      document['month'] = flight.MONTH
      document['day_of_month'] = flight.DAY_OF_MONTH
      document['day_of_week'] = flight.DAY_OF_WEEK
      document['origin'] = flight.ORIGIN.upper()
      document['origin_info'] = json_airport_info(flight.YEAR, flight.ORIGIN, 
                                                  flight.ORIGIN_CITY_NAME, 
                                                  flight.ORIGIN_STATE_NM)
      document['destination'] = flight.DEST.upper()
      document['destination_info'] = json_airport_info(flight.YEAR, flight.DEST, 
                                            flight.DEST_CITY_NAME, 
                                            flight.DEST_STATE_NM)
      document['distance'] = int(flight.DISTANCE)
      document['distance_group'] = flight.DISTANCE_GROUP
    
    # Append all flight information to the list
    flights.append(json_single_flight(flight))

  # add the list of all flight to the document
  document['flights'] = flights
  
  # TODO: upload to mongo db

  # ALTERNATIVE CODE FOR TESTING WITHOUT MONGO UPLOAD, 
  # please note that this method is computationally expensive because create
  # a milions of files!

  """
  # writing json to file
  with open('./Json-schema/sample/prova' + str(i) + '.json', 'w') as fp:
    json.dump(document, fp)
    
    # alternative way for writing dictionary to json file if dumps is already done   
    # fp.write(json_data)
    i = i+1
  """

### Validate or download result

In [ ]:
# If you want to validate a json file uncomment the following lines
"""
# Try to validate a sample json file (write with method before if you want to check all files)
f = open('./Json-schema/sample/prova.json',) 
data = json.load(f) 
# load json schema for validate files
json_schema = json.loads(open('./Json-schema/JsonFlightSchema.json').read())
json_validator(json_schema, data)
"""

In [ ]:
# If you have used the testing version and you want to download the zip 
# file uncomment the following line
# !zip -r json_file.zip ./Json-schema/sample3/ 